Draws a circle on the detected position of the tennis ball.

This script takes json files as input.



In [1]:
import cv2
import json
import pandas as pd
import numpy as np
import os
from IPython.display import display, HTML, Video

In [2]:
BOX_COLOR = (0, 255, 255)  # Green color in BGR (Blue, Green, Red)
BOX_THICKNESS = 3        # Thickness of the bounding box line

In [3]:
def process_video_with_boxes(video_path, json_path, output_path):
    """
    Reads a video, draws bounding boxes based on JSON coordinates, and saves the result.
    """
    print(f"Loading data from: {json_path}")

    try:
        with open(json_path, 'r') as f:
            tracking_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_path}")
        return
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON file at {json_path}")
        return

    df = pd.DataFrame(tracking_data).set_index('frame')

    print(f"Tracking data loaded for {len(df)} frames.")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    current_frame_id = 0

    radius = 15

    print(f"Processing video ({frame_width}x{frame_height} at {fps:.2f} FPS)...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # End of video stream

        current_frame_id += 1

        if current_frame_id in df.index:
            try:
                box_data = df.loc[current_frame_id]

                if pd.isna(box_data['x']) or pd.isna(box_data['y']):
                    print(f"Warning: Skipping frame {current_frame_id} due to NaN coordinates.")
                    continue # Skip to the next frame without drawing

                x = int(box_data['x'])
                y = int(box_data['y'])

                cv2.circle(
                frame,
                (x, y),  # Center point (cx, cy)
                radius,
                BOX_COLOR,
                BOX_THICKNESS
                )

                """cv2.putText(frame,
                            f"Frame: {current_frame_id}",
                            (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            BOX_COLOR,
                            BOX_THICKNESS - 1)"""

            except KeyError as e:
                print(f"Warning: Missing key in JSON for frame {current_frame_id}: {e}")
            except ValueError as e:
                print(f"Warning: Could not convert coordinate to integer for frame {current_frame_id}: {e}")

        out.write(frame)

    cap.release()
    out.release()

    print(f"Processing complete! Video saved to: {output_path}")
    print(f"Frames processed: {current_frame_id}")

In [4]:
input_video = '/content/test-video.mp4'
json_file = '/content/player_ball_video_interpolated.json'
output_video = '/content/interpolated-ball-position-video.mp4'

In [5]:
if not os.path.exists(input_video):
    print(f"\nATTENTION: Please place your video file named '{input_video}' in the same directory as this script.")
    print("Execution stopped. After placing the video, run the script again.")
else:
    process_video_with_boxes(input_video, json_file, output_video)

Loading data from: /content/player_ball_video_interpolated.json
Tracking data loaded for 284 frames.
Processing video (1920x1080 at 50.00 FPS)...
Processing complete! Video saved to: /content/interpolated-ball-position-video.mp4
Frames processed: 301
